## В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Знайти датасет ви можете за [посиланням](https://drive.google.com/file/d/1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO/view?usp=share_link).

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [11]:
from pathlib import Path
from urllib.request import urlretrieve
import zipfile

URL = "https://drive.usercontent.google.com/download?id=1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO&export=download"

FILE_NAME = Path('content/data.zip')
FOLDER_NAME = Path('content')

# urlretrieve(URL, FILE_NAME)

with zipfile.ZipFile(FILE_NAME, 'r') as zipfp:
    zipfp.extractall(FOLDER_NAME)

In [17]:
p = FOLDER_NAME.glob('**/*.csv')
files = [x for x in p if x.is_file()]

6462

In [19]:
# df = pd.read_csv(URL, header=0)
# if len(df) == 0:
#     print("Error: file not found")
#     raise ValueError

# df.head()

def read_csv(path):
    df = pd.read_csv(path, header=0)
    if len(df) == 0:
        print("Error: file not found")
        raise ValueError
    return df

In [28]:
print([item for item in FOLDER_NAME.glob('**/*') if item.is_dir() and item.name != FOLDER_NAME.name])

print(files[0])
df = read_csv(files[0])
df

[WindowsPath('content/data'), WindowsPath('content/data/idle'), WindowsPath('content/data/running'), WindowsPath('content/data/stairs'), WindowsPath('content/data/walking')]
content\data\idle\idle-1.csv


,accelerometer_X,accelerometer_Y,accelerometer_Z
0,1.000776,4.616021,8.576031
1,0.718261,4.209007,8.446744
2,-0.909797,-0.282516,9.203311
3,5.099650,0.148441,8.418014
4,1.762132,-0.162806,9.251195
5,-0.086191,-0.148441,9.787497
6,-0.062249,-0.191536,9.773131
7,-0.057461,-0.114922,9.763555
8,-0.124498,-0.162806,9.782708
9,-0.138864,-0.124498,9.768343


## Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній [статті](https://drive.google.com/file/d/1-18YEmp0YjV3hN9iI8J1i_FWd55HFwOK/view?usp=share_link).

## Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.

## Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою. Використайте метод classification report для порівняння.